In [1]:
xai_result = """
You are a Cyber Security Analyst.
Payload is GET /etc/passwd?SELECT%20FROM*table HTTP/1.1 UserAgent: sqlmap.
Based on the payload above, the AI predicted an attack during an attack/normal.
Below are the top 3 features for feature importance.
First feature is payload_word_comb_01, that feature means an expression for the case where a combination of attack related keywords is included in the payload.
Feature importance(%) is 33.50 and  predict direction is attack and AI attack detect keyword is "etc/passwd".
Second feature is payload_useragent_comb, that feature means an expression for the case where the payload contains a malicious user_agent.
Feature importance(%) is 30.07 and  predict direction is attack and AI attack detect keyword is "UserAgent: sqlmap".
Third feature is payload_sql_comb_01, that feature means an expression for the case where the payload includes a combination of keywords related to SQL-Injection.
Feature importance(%) is 29.79 and  predict direction is attack and AI attack detect keyword is "SELECT%20FROM".
Explain it in 3 sentences based AI attack detect keyword from first, second, third features in feature importance above.
"""

In [2]:
import torch
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

# model 디바이스 설정
# model.device = model.to(device)

In [4]:
summarizer = pipeline("summarization", model = model, tokenizer = tokenizer, device = device, do_sample = True,
                                    min_length = 100, max_length = 300)

In [5]:
import os
save_model_path = 'chatgpt_context/summarization'
model_dir = os.path.join(save_model_path, 'pytorch_distilbart_summa.pt')

# 모델 가중치 및 토크나이저 별도 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'tokenizer_state_dict': tokenizer.save_pretrained(save_model_path),
}, model_dir)

In [6]:
# 모델 호출
load_model = torch.load(model_dir)
model.load_state_dict(load_model['model_state_dict'])
tokenizer = AutoTokenizer.from_pretrained(save_model_path)

# load_model.eval()
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((102

In [7]:
model.config

BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": tr

In [8]:
model.device

device(type='cpu')

In [9]:
summarizer_result = summarizer(xai_result)

Your max_length is set to 300, but you input_length is only 274. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=137)


In [10]:
summarizer_result

[{'summary_text': ' The payload is GET /etc/passwd? SELECT%20FROM*table HTTP/1.1 UserAgent: sqlmap . The AI predicted an attack during an attack/normal . The payload includes a combination of attack related keywords . The feature importance of the payload is 33.50 and  predict direction is attack and AI attack detect keyword is "etc/Passwd". The payload also includes a malicious user agent . The result is the result of an attack on the payload .'}]

In [ ]:
output = """
The payload is GET /etc/passwd? SELECT%20FROM*table HTTP/1.1 UserAgent: sqlmap .
The AI predicted an attack during an attack/normal . 
The payload includes a combination of attack related keywords . 
The feature importance of the payload is 33.50 and  predict direction is attack and AI attack detect keyword is "etc/Passwd". 
The payload also includes a malicious user agent . The result is the result of an attack on the payload .
"""